## Scopes

This will be a very brief intro on scopes.  Python, unlike most modern languages is not lexically scoped.

> What does `lexically scoped` mean?  Lexical means essentially, words, but it has to do with "boundaries" of your code.  
> In many languages with curly braces, the braces define a scope.  Variables or other symbols introduced in that scope  
> (eg the curly braces) only last inside the brace section.

Python is not like that

- Indentation does **not** create a new scope in python
- a scope contains namespace, which _binds_ a `symbol` to a value 
    - scopes -> namespaces -> symbols -> values

## Namespaces and symbols

Many people who program tend to forget or never truly learn what a `symbol` in a language is.  For example consider this
one-liner:

```python
path = "/tmp/some-data"
```

As programmers, we think in our heads:

```
path is a variable that has the value '/tmp/some-data'`
```

But that's wrong in several ways.  More precisely:

- `path` is a symbol...
- in the global scope...
- where `path` is a look up (key) into a table...
- and the value is a reference to an object.  
- The object that symbol points to:
    - has a memory location 
    - and a type.  
- The type of the object determines how we get the actual value it refers to

The next cell shows several variables which live in different namespaces

In [ ]:
from pprint import pformat, pprint
lookup_val = "hello"  # Global scoped.  Global is actually module-scoped (only builtins are truly global)

def lookup():
    lookup_val = 100
    enclosing = "enclosed variable"
    print(f"lookup() locals: {locals()}")
    #pprint(globals())
    def inner():
        enclosing = "local variable"
        print(f"in inner(): lookup_val = {lookup_val}")
        print(f"inner() locals: {locals()}")
        print(f"inner() globals: {pformat(globals())}")
    print(f"In lookup: lookup_val = {lookup_val}")
    return inner

inn = lookup()
inn()  # which lookup_val is inner going to use?
print(f"In global scope: lookup_val is {lookup_val}") # it's still 10

## Mutable vs Immutable captures

One of the more important takeaway lessons is that 

- **everything** in python is a reference (even an int or float is a reference not _just_ data)
- in python, you **always** pass by reference
    - However, some references are immutable

> Thanks to garbage collected languages many software engineers who didn't grow up with C(++) or learned a newer system  
> programming language like rust no longer know what _pass by reference_ or _pass by copy/value_ means.  This will be a  
> a huge presentation I will give in the future, but know for now that _pass by reference_ means you are directly  
> operating on an object, and not a copy of an object

In the code above, `lookup_val` is an int, and in python, `int` types are one of the few immutable types.  So, even  
though you can pass an int like `lookup_val` to a function, python will not change the value it has _in memory_

In [7]:
# This won't do what you think.  It does **not** change whatever `age` is even though we pass by reference.  This is
# because an int type is an immutable reference.
def wont_change_arg(age: int):
    age = 100

current_age = 20
wont_change_arg(current_age)
print(current_age)

20


### Mutable references

So what happens to _mutable_ references (which are most things in python)?

In [8]:
# Now let's declare a mutable reference
a_list = [1, 2]

# Because a_list is a mutable reference and it's in the global (top-level) scope, the call here will mutate a_list
def captured_a_list(val: int):
    a_list.append(val)
    # print(locals())

captured_a_list(10)  # what do you think a_list is now?

In [9]:
# Since we called captured_a_list(10), a_list should be [1, 2, 10] now
print(a_list)

def replacer(some_list: list[int]):
    print(f"id of some_list before assignment = {hex(id(some_list))}")
    some_list = []
    print(f"id of some_list after assignment is now = {hex(id(some_list))}")
    print(f"but id of a_list is still = {hex(id(a_list))}")

print(f"id of a_list is {hex(id(a_list))}")
replacer(a_list) # this is like lookup(), it creates a new binding, leaving the original alone
print(a_list)

[1, 2, 10]
id of a_list is 0x7fea4c16ad40
id of some_list before assignment = 0x7fea4c16ad40
id of some_list after assignment is now = 0x7fea4c169440
but id of a_list is still = 0x7fea4c16ad40
[1, 2, 10]


## Wait...what happened?

So can anyone explain what happened?

Bueller?

Bueller?

If you understood what happened there, great!  If not, it's too deep of a topic to talk about for now.  Wait for the
presentation on a deep dive on how memory in computers works!  This will be a standalone talk since it will not be
exclusive to python, and will benefit anyone.

## Python scope lookup

There are 4 kinds of scope in python in this order of lookup

- local
- enclosing
- global
- builtin

```
+-----------------------------------+
| builtins                          |
|                                   |
| +-------------------------------+ |
| | globals                       | |
| |                               | |
| | +---------------------------+ | |
| | | enclosing/non-locals      | | |
| | |                           | | |
| | | +----------------------+  | | |
| | | | locals               |  | | |
| | | +----------------------+  | | |
| | +---------------------------+ | |
| +-------------------------------+ |
+-----------------------------------+
```

You might wonder what is the difference between local and enclosing?

## The global and nonlocal keywords

Though rarely used, sometimes it is necessary to mark a variable in a non top-level scope (ie, declared inside of a 
function, method, or class) as being either `global` or `nonlocal`.

Let's look at an example:

In [ ]:
def globalizer():
    global foo
    foo = "testing"

globalizer()
# Where did `foo` come from?  It got inserted into the global namespace through the `global` keyword
print(f"foo = {foo}")

def change_foo():
    foo = "hello"  # What do you think this will print?

## What will change_foo print?

In [ ]:
change_foo()
print(f"foo did not change and is still = {foo}")

# So how DO you change it then?

### How to "change" an immutable global

As we can see, we tried to reassign the `global` variable foo inside of `change_foo` function, but it didn't work.  Is
there a way to make it work?

There is, but you have to add the `global` keyword to the function that will mutate the global variable.

In [40]:
name = "sean"  # A global (aka module scoped) variable since it is not inside a function or class
name_copy = name
print(f"id of name = {hex(id(name))}, id of name_copy = {hex(id(name_copy))}")

# def non_global_rename(to: str):
#     name = to  # Question: what scope does `name` live in?

# non_global_rename("john")
# print(f"without `global name`, name is still = {name}")

def rename(to: str):
    global name
    name = to
    print(f"id of name is now {hex(id(name))}")

print(f"Before call to rename, name = {name}")
rename("john") # because we use global, we reassign it
print(f"after call to rename, name is now = {name}")
print(f"name_copy is {name_copy} and its id is {hex(id(name_copy))}")

id of name = 0x7fea364774b0, id of name_copy = 0x7fea364774b0
Before call to rename, name = sean
id of name is now 0x7fea3645f6b0
after call to rename, name is now = john
name_copy is sean and its id is 0x7fea364774b0


In [21]:
def change_foo():
    global foo
    foo = "hello"  # What do you think this will print?

change_foo()
print(f"foo is now {foo}")

foo is now hello


## Extra Credit

I won't go over these, because I will need to talk more about memory.  But feel free to play around with these examples
and step through the code to see if you can understand what is going on.

In [33]:
# What would happen here?  Let's redefine
a_list = [1, 2]
print(f"id of a_list = {hex(id(a_list))}")
print("====================")

def replacer_1(some_list: list[int]):
    print(f"id of some_list before assignment = {hex(id(some_list))}")
    global a_list
    some_list = a_list
    print(f"id of some_list after assignment is now = {hex(id(some_list))}")
    some_list = []
    print(f"but id of a_list is now = {hex(id(a_list))}")

beginning_val = list()
print(f"id of beginning_val = {hex(id(beginning_val))}")
replacer_1(beginning_val)
print(beginning_val, a_list)

def replacer_2(arr: list[int]):
    print(f"id of arr before assignment = {hex(id(arr))}")
    a_list = arr
    print(f"id of arr after assignment is now = {hex(id(arr))}")
    print(f"but id of a_list is now = {hex(id(a_list))}")

print("====================")

beginning_val = list()
print(f"id of beginning_val = {hex(id(beginning_val))}")
replacer_2(beginning_val)
print(beginning_val, a_list)

print("====================")
print(f"id of a_list is now {hex(id(a_list))}")
a_list

id of a_list = 0x7fea4fbc4480
id of beginning_val = 0x7fea4c28bfc0
id of some_list before assignment = 0x7fea4c28bfc0
id of some_list after assignment is now = 0x7fea4c28ad00
but id of a_list is now = 0x7fea4fbc4480
[] [1, 2]
id of beginning_val = 0x7fea4c28b7c0
id of arr before assignment = 0x7fea4c28b7c0
id of arr after assignment is now = 0x7fea4c28b7c0
but id of a_list is now = 0x7fea4c28b7c0
[] [1, 2]
id of a_list is now 0x7fea4fbc4480


[1, 2]

## Another example of a Mutable variable

Most python types are mutable, to the point that most documentation will only specify when a type is **not** mutable.
Major examples are `str`, `int`, `float` and `tuple` and anything inheriting from those classes.

In [34]:
from datetime import datetime


build_to_date = {}

def add_to_map(key: int):
    build_to_date[key] = datetime.now()
    return build_to_date

In [35]:
import time


for i in range(3):
    print(add_to_map(i))
    time.sleep(1)

{0: datetime.datetime(2023, 11, 6, 1, 4, 18, 807238)}
{0: datetime.datetime(2023, 11, 6, 1, 4, 18, 807238), 1: datetime.datetime(2023, 11, 6, 1, 4, 19, 807376)}
{0: datetime.datetime(2023, 11, 6, 1, 4, 18, 807238), 1: datetime.datetime(2023, 11, 6, 1, 4, 19, 807376), 2: datetime.datetime(2023, 11, 6, 1, 4, 20, 807585)}


## More scoping gotchas

This is a common source of confusion for new pythonistas
score = 50 # try setting this to 50

In [ ]:
def grader(check: int):
    # dont have to create a "default" grade here,  grade is scoped to the entire function
    # however, try changing score to 50, and commenting out the marked lines
    if check > 90:
        grade = "A"
    elif check > 80:
        grade = "B"
    elif check > 70:
        grade = "C"
    elif check > 60:
        grade = "D"
    #else:              # try commenting this out
    #    grade = "F"    # and this
    return grade


grader(score)

## Higher order functions

Part of the reason that namespaces and scopes were covered, is to help better understand how higher order functions
work.  In python, functions are:

- first class citizens (unlike Java or C/C++) 
- can be used as arguments to functions or returned from functions
- capture their namespace, namely, their local and enclosing scoped namespaces
    - In a sense, you can think of functions as mini-objects that contain state.

We will next cover:

- nested functions
- functions as return values
- functions as arguments

In [ ]:
# A common use case of a nested function, is to create a "private" function that handles some functionality

import functools as ft
from random import randint
from typing import Callable, Generator, Sequence

# Example of a function that returns a function
# Question: what scope is `num` in? (hint: from whose perspective?)
def die_pool(num: int):
    if num < 1:
        raise Exception(f"num must be greater than 0")
    def inner(size: int):
        if size < 2:
            raise Exception("size must be >= 2")
        if size > 1000:
            raise Exception("Only up to d1000 is possible")
        # Returns a generator, which I will cover when we go over the Iterator protocol.  Think of it as a lazy list
        return (randint(1, size) for _ in range(num))
    return inner

# Example of a nested function
def die_roll_v1(
    num: int, 
    size: int, 
    target: int
):
    """Rolls num of size dice.  Returns a 2 element tuple of rolls < target, and >= target

    Parameters
    ----------
    num : int
        number of dice to roll
    size : int
        size of the die
    target : int
        A number from 1 - size
    """
    if target < 1 or target > size:
        raise Exception("target must be: 1 <= target <= size")

    # Example of a nested function.  This is one way of making something "private" in python.  Technically, you can 
    # still access 
    def filter(acc: tuple[list[int], list[int]], next: int):
        if next >= target:
            acc[1].append(next)
        else:
            acc[0].append(next)
        return acc
    
    roll = die_pool(num)(size)
    result = ft.reduce(filter, roll, ([], []))
    return sorted(result[0]), sorted(result[1])

In [ ]:
failed, succeeded = die_roll_v1(6, 20, 12)
print(failed, succeeded)

In [ ]:
# This is an example of a function as a parameter.  Here, we are passing in `roll`, which is a function that takes no
# arguments, and returns a Generator.  I will go more in depth in Generators when we cover the Iterator protocol.  For
# now, think of it as a lazy list that yields one element at a time
def die_roll_v2(
    pool: Callable[[], Generator[int, None, None]], 
    target: int
):
    """Takes a roll of dice (list of int) and finds all values >= to target.  If exploding is not None, if any value
    is >= to exploding, roll another die (recursively)

    []

    Parameters
    ----------
    roll : list[int]
        _description_
    target : int
        _description_
    """

    def filter(acc: tuple[list[int], list[int]], next: int):
        if next >= target:
            acc[1].append(next)
        else:
            acc[0].append(next)
        return acc
    
    roll = pool()
    result = ft.reduce(filter, roll, ([], []))
    return sorted(result[0]), sorted(result[1])

In [ ]:
p5d20 = lambda: die_pool(5)(20)
p7d20 = lambda: die_pool(7)(20)
failed, successes = die_roll_v2(p7d20, 12)
print(failed, successes)

In [ ]:
# Example of a stateful closure.  And this is one reason I spent all that time going over scopes and namespaces

def find(match: str):
    count = 0

    def search(text: str):
        nonlocal count  # Try commenting this out
        if match in text:
            count += 1
        return count

    return search

matcher = find("sean")
text = [
    "Hi sean how are you?",
    "Did sean do his work today?",
    "Why not?",
    "Excuses excuses",
    "Get cracking sean!"
]
for line in text:
    count = matcher(line)
    print(f"found {count} matches so far")
